In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import pandas as pd
import openai
import os
import json
from gpt3_sandbox.api.gpt import GPT
from gpt3_sandbox.api.gpt import Example
from pandasql import sqldf
from tqdm import tqdm
import numpy as np
from GptPrompter import *
from AutoReasoner import *

dataset = pd.read_csv('./dataset/WikiTableQuestions/data/pristine-unseen-tables.tsv', sep='\t')

In [70]:
i = 3
print("utterance = ", dataset.iloc[i]["utterance"])
print("gold answer = ", dataset.iloc[i]["targetValue"])
df = pd.read_csv(f'./dataset/WikiTableQuestions/{dataset.iloc[i]["context"]}', sep=',')

table_text = table_formater(df, key_val_pairs=False)
print(table_text)

utterance =  alfie's birthday party aired on january 19. what was the airdate of the next episode?
gold answer =  January 26, 1995
[HEAD]: series_#|season_#|title|notes|original_air_date
-------------------------------------------------------
[ROW] 1: 1|1|\The Charity\""|Alfie, Dee Dee, and Melanie are supposed to be helping their parents at a carnival by working the dunking booth. When Goo arrives and announces their favorite basketball player, Kendall Gill, is at the Comic Book Store signing autographs, the boys decide to ditch the carnival. This leaves Melanie and Jennifer to work the booth and both end up soaked. But the Comic Book Store is packed and much to Alfie and Dee Dee's surprise their father has to interview Kendall Gill. Goo comes up with a plan to get Alfie and Dee Dee, Gill's signature before getting them back at the local carnival, but are caught by Roger. All ends well for everyone except Alfie and Goo, who must endure being soaked at the dunking booth.|October 15, 19

In [53]:
prompt = f"""
Convert the following table into multiple text descriptions. 
Generate comprehensive, self-containing descriptions of the table.

The table is:
{table_text}

The text descriptions are: ```
"""

# model = "davinci-codex-002-msft"
model = "text-davinci-002"
import dotenv
config = dotenv.dotenv_values(".env")
openai.api_key = config['OPENAI_API_KEY_ms']

table_text = openai.Completion.create(engine=model,
                        prompt=prompt,
                        max_tokens=2048,
                        temperature=0,
                        top_p=1,
                        frequency_penalty=0,
                        n=1,
                        stream=False,
                        stop='```').choices[0].text

In [54]:
print(table_text)

1st: Alejandro Valverde (ESP) - Caisse d'Epargne - 5h 29' 10",40"
2nd: Alexandr Kolobnev (RUS) - Team CSC Saxo Bank - s.t.
3rd: Davide Rebellin (ITA) - Gerolsteiner - s.t.
4th: Paolo Bettini (ITA) - Quick Step - s.t.
5th: Franco Pellizotti (ITA) - Liquigas - s.t.
6th: Denis Menchov (RUS) - Rabobank - s.t.
7th: Samuel Sánchez (ESP) - Euskaltel-Euskadi - s.t.
8th: Stéphane Goubert (FRA) - Ag2r-La Mondiale - + 2",5"
9th: Haimar Zubeldia (ESP) - Euskaltel-Euskadi - + 2",3"
10th: David Moncoutié (FRA) - Cofidis - + 2",1"


In [21]:
prompt = f"""
Answer the following question accoding to the descriptions below: {dataset.iloc[i]["utterance"]}.

{table_text}

The answer is: ```
"""

ans = openai.Completion.create(engine=model,
                        prompt=prompt,
                        max_tokens=2048,
                        temperature=0,
                        top_p=1,
                        frequency_penalty=0,
                        n=1,
                        stream=False,
                        stop='```').choices[0].text

In [22]:
ans

'\nThe attendance for the Toros Mexico vs. Monterrey Flash game was 118.'

In [17]:
import time

# model = 'text-davinci-002'
# model = 'text-davinci-003'
model = 'davinci-codex-002-msft'

ft = fasttext.load_model('/mnt/idm_automapping/cc.en.300.bin')

def parallel_gpt_answer_dynamic_few_shot_func(i):
    try:
        prompter = CodexAnswerNLTable(test_df.iloc[i]['id'], 
                             test_df.iloc[i]['utterance'], 
                             test_df.iloc[i]['context'], 
                             test_df.iloc[i]['targetValue'], 
                             base_path='./dataset/WikiTableQuestions/')
        prompter.model = model
        prompter._set_table_nl_desc(table_nl_desc_train, table_nl_desc_test)
        prompter._gen_NN_demo(training_data_autoreasoning, all_demo_embeddings, ft, demo_num=3)
        prompter._gen_gpt_prompt()
        prompter._get_gpt_prediction()
        if prompter.gpt_error is None:
            prompter._evaluate_result(verbose=False)
        log = prompter._log_dict()
    except Exception as e:
        log = {
            'id': test_df.iloc[i]['id'],
            'uncaught_err': str(e)
        }
    if model == 'text-davinci-003':
        time.sleep(5)
    return log

training_data_autoreasoning = json.load(open(f'./dataset/WikiTableQuestions/reasonings/GptReasoner_training.json', 'r'))
all_demo_embeddings = None
print("Loading embeddings")
for train_q in tqdm(training_data_autoreasoning):
    if all_demo_embeddings is None:
        all_demo_embeddings = get_utterance_embedding(train_q['utterance'], ft).reshape([-1, 1])
    else:
        all_demo_embeddings = np.append(all_demo_embeddings, get_utterance_embedding(train_q['utterance'], ft).reshape([-1, 1]), axis=1)

test_df = pd.read_csv(f'./dataset/WikiTableQuestions/data/pristine-unseen-tables.tsv', sep='\t', on_bad_lines='skip')
table_nl_desc_train = json.load(open('Table2Text_training.json', 'r'))
table_nl_desc_test = json.load(open('Table2Text_prestine-unseen-tables.json', 'r'))

maxLimit = float('inf')
n_threads = 20
from joblib import Parallel, delayed
res = Parallel(n_jobs=n_threads, require='sharedmem')(delayed(parallel_gpt_answer_dynamic_few_shot_func)(i) for i in tqdm(range(min(maxLimit, test_df.shape[0]))))
json.dump(res, open(f'./dataset/WikiTableQuestions/results/GptAnswerNLTable_{model}_fewShot_top3cos_results_pristine-unseen-tables.json', 'w'))

Loading embeddings


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4344/4344 [17:28<00:00,  4.14it/s]


In [65]:
table_nl_desc_train = json.load(open('Table2Text_training.json', 'r'))
table_nl_desc_test = json.load(open('Table2Text_prestine-unseen-tables.json', 'r'))

In [66]:
table_nl_desc_test_tuned = table_nl_desc_test.copy()

In [28]:
i = 24
print(table_nl_desc_test[i]['table_text'])
print(table_nl_desc_test[i]['nl_text'])

[HEAD]: Match|Date|Venue|Opponents|Score
----------------------------------------
[ROW] 1: GL-B-1|2008..|[[]]|[[]]|-
[ROW] 2: GL-B-2|2008..|[[]]|[[]]|-
[ROW] 3: GL-B-3|2008..|[[]]|[[]]|-
[ROW] 4: GL-B-4|2008..|[[]]|[[]]|-
[ROW] 5: GL-B-5|2008..|[[]]|[[]]|-
[ROW] 6: GL-B-6|2008..|[[]]|[[]]|-
[ROW] 7: Quarterfinals-1|2008..|[[]]|[[]]|-
[ROW] 8: Quarterfinals-2|2008..|[[]]|[[]]|-

1. Match 1: 2008. [[]] [[]] -
2. Match 2: 2008. [[]] [[]] -
3. Match 3: 2008. [[]] [[]] -
4. Match 4: 2008. [[]] [[]] -
5. Match 5: 2008. [[]] [[]] -
6. Match 6: 2008. [[]] [[]] -
7. Quarterfinals 1: 2008. [[]] [[]] -
8. Quarterfinals 2: 2008. [[]] [[]] -


In [22]:
table_nl_desc_test_tuned[i]['nl_text']= """1. Match GL-B-1: 2008. [[]] [[]] -
2. Match GL-B-2: 2008. [[]] [[]] -
3. Match GL-B-3: 2008. [[]] [[]] -
4. Match GL-B-4: 2008. [[]] [[]] -
5. Match GL-B-5: 2008. [[]] [[]] -
6. Match GL-B-6: 2008. [[]] [[]] -
7. Quarterfinals 1: 2008. [[]] [[]] -
8. Quarterfinals 2: 2008. [[]] [[]] -"""

In [32]:
prompt = f"""
A database table is shown as follows: Component|Model 01|Model 01+|Model 02|Model e2|Model 2+ (Pre-production)|model 03 (China Copy) ------------------------------------------------------------------------------------------------------ [ROW] 1: CPU|Transmeta Crusoe 1 GHz|Transmeta Crusoe 1 GHz|Via C7M ULV 1.6 GHz|Via C7M ULV 1.6 GHz|Intel Atom Z540 1.86 GHz|Intel atom Z550 1.2 GHz dual core [ROW] 2: RAM|256MB DDR|512MB DDR|1GB DDR2|1GB DDR2|2GB DDR2|2GB DDR2 [ROW] 3: Hard Drive|20GB HDD|30GB HDD|120GB HDD or 64GB SSD|120GB HDD or 64GB SSD|120GB HDD or 64GB SSD|120GB HDD or 64GB SSD [ROW] 4: Display|5\ Transflective 800x480 LCD"|5\ Transflective 800x480 LCD"|5\ 800x480 LCD"|5\ 800x480 LCD"|5\ active matrix 800x480 OLED"|4.8\ active matrix 1024X600 OLED" [ROW] 5: USB|1.1|2.0|2.0|2.0|2.0|2.0 [ROW] 6: Wi-Fi|802.11b|802.11b|802.11a/b/g|802.11a/b/g|802.11a/b/g|802.11a/b/g/n [ROW] 7: WWAN|nan|nan|EVDO from Sprint or Verizon|HSDPA|EV-DO and HSPA|HSDPA 3G [ROW] 8: Bluetooth|1.1|1.1|2.0|2.0|2.0|2.1 [ROW] 9: Wacom|Yes|Yes (Improved accuracy)|Yes|Yes|Yes|unknown [ROW] 10: GPU|Silicon motion Lynx 3DM+|Silicon motion Lynx 3DM+|VIA VX700|VIA VX700|Intel GMA500|Intel GMA500 [ROW] 11: Removable Battery|4,000 mAh or 8,000 mAh|4,000 mAh or 8,000 mAh|4,500 mAh or 9,000 mAh|4,500 mAh or 9,000 mAh|4,500 mAh or 9,000 mAh|4,500 mAh or 9,000 mAh [ROW] 12: Battery Type|lithium polymer|Lithium Polymer|lithium ion polymer|lithium ion polymer|lithium ion polymer|unknown [ROW] 13: Docking Cable|USB 1.1|USB 2.0|replaced by dongle or dock|replaced by dongle or dock|replaced by dongle or dock|replaced by dongle or dock [ROW] 14: Dock|Zinc stand|Zinc stand|Gloss Black w/ optical drive|Gloss Black w/ optical drive|Gloss Black w/ optical drive|Gloss Black w/ optical drive [ROW] 15: Ethernet|10BaseT|100BaseT|100BaseT|100BaseT|100BaseT|100BaseT [ROW] 16: Dongle|nan|nan|RJ45 & VGA|RJ45 & VGA|unknown|unknown [ROW] 17: Keyboard|57 key|57 key|58 key|58 key|58 key|58 key [ROW] 18: Weight|397g|397g|413g*|413g*|413g*|426g* Answer the following question based on the data above: "what component comes after bluetooth?". The answer is: ```Wacom```. A database table is shown as follows: 1. The Canada-United States border is the location of the Canada-United States border. ND 18 – Cavalier, Larimore is the interesting highway. PTH 30 begins at this point. 2. The R.M. of Rhineland is located in Gretna. PR 243 east – Emerson is the interesting highway. PR 243 west concurrence begins at this point. 3. The R.M. of Rhineland is located in nan. PR 243 west – Blumenfeld is the interesting highway. PR 243 west concurrence ends at this point. 4. The R.M. of Rhineland is located in nan. PR 421 east – Sommerfeld is the interesting highway. 5. The R.M. of Rhineland is located in Altona. PR 201 west – Brown, Windygates, Snowflake is the interesting highway. PR 201 east concurrence begins at this point. 6. The R.M. of Rhineland is located in nan. PR 201 east – Letellier, Dominion City, Stuartburn is the interesting highway. PR 201 east concurrence ends at this point. 7. The R.M. of Rhineland is located in Rosenfeld. PTH 14 – Morden, Winkler, Plum Coulee, Morris is the interesting highway. PR 332 north – Lowe Farm, Brunkild, Starbuck is the interesting highway. PTH 30 ends at this point and the highway continues as PR 332. Answer the following question based on the data above: "what location comes after gretna?". The answer is: ```Altona```. A database table is shown as follows: 1. The Complete Series 1: This two-disc set includes all eight episodes from the first series of The Two Ronnies, originally aired in 1971. The DVD was released in the UK on 30 April 2007 and in Australia on 4 July 2007. 2. The Complete Series 2: This two-disc set includes all eight episodes from the second series of The Two Ronnies, originally aired in 1972. The DVD was released in the UK on 2 July 2007 and in Australia on 8 May 2008. 3. The Complete Series 3: This two-disc set includes all eight episodes from the third series of The Two Ronnies, originally aired in 1973-1974. The DVD was released in the UK on 17 March 2008 and in Australia on 5 March 2009. 4. The Complete Series 4: This two-disc set includes all eight episodes from the fourth series of The Two Ronnies, originally aired in 1975. The DVD was released in the UK on 19 May 2008 and in Australia on 6 August 2009. 5. The Complete Series 5: This two-disc set includes all eight episodes from the fifth series of The Two Ronnies, originally aired in 1976. The DVD was released in the UK on 15 June 2009 and in Australia on 4 March 2010. 6. The Complete Series 6: This two-disc set includes all eight episodes from the sixth series of The Two Ronnies, originally aired in 1977-1978. The DVD was released in the UK on 22 February 2010 and in Australia on 5 August 2010. 7. The Complete Series 7: This two-disc set includes all eight episodes from the seventh series of The Two Ronnies, originally aired in 1978-1979. The DVD was released in the UK on 17 May 2010 and in Australia on 3 March 2011. 8. The Complete Series 8: This two-disc set includes all eight episodes from the eighth series of The Two Ronnies, originally aired in 1980. The DVD was released in the UK on 6 September 2010 and in Australia on 2 June 2011. 9. The Complete Series 9: This two-disc set includes all eight episodes from the ninth series of The Two Ronnies, originally aired in 1981-1982. The DVD was released in the UK on 28 February 2011 and in Australia on 1 March 2012. 10. The Complete Series 10: This six-disc set includes all six episodes from the tenth series of The Two Ronnies, originally aired in 1983-1984. The DVD was released in the UK on 9 May 2011 and in Australia on 5 September 2012. 11. The Complete Series 11: This five-disc set includes all five episodes from the eleventh series of The Two Ronnies, originally aired in 1985. The DVD was released in the UK on 25 July 2011 and in Australia on 3 April 2013. 12. The Complete Series 12: This six-disc set includes all six episodes from the twelfth series of The Two Ronnies, originally aired in 1985-1986. The DVD was released in the UK on 26 September 2011 and in Australia on 7 August 2013. 13. The Christmas Specials: This four-disc set includes four Christmas specials from The Two Ronnies, originally aired in 1973, 1982, 1984, and 1987. The DVD was released in the UK on 29 October 2007 and in Australia on 6 November 2008. 14. The Complete Collection: This 27-disc set includes all 96 episodes from The Two Ronnies, originally aired between 1971 and 1987, as well as the 2010 special. The DVD was released in the UK on 24 September 2012 and in Australia in 2013. 15. Two Ronnies In Australia: This two-disc set includes all six episodes from The Two Ronnies' Australian special, originally aired in 1986. The DVD was released in the UK on 28 June 2008. 16. The Best of...Volume 1: This one-disc compilation set includes episodes from the first 14 series of The Two Ronnies, originally aired between 1971 and 1984. The DVD was released in the UK on 1 October 2001 and in Australia on 4 March 2002. 17. The Best of...Volume 2: This one-disc compilation set includes episodes from the first 15 series of The Two Ronnies, originally aired between 1971 and 1985. The DVD was released in the UK on 29 September 2003 and in Australia on 17 March 2003. Answer the following question based on the data above: "what comes immediately after complete series 11?". The answer is: ```Complete Series 12```. A database table is shown as follows:{table_nl_desc_test_tuned[i]['nl_text']} Answer the following question based on the data above: "what Match comes after gl-b-5?". The answer is: ```
"""

ans = openai.Completion.create(engine=model,
                        prompt=prompt,
                        max_tokens=2048,
                        temperature=0,
                        top_p=1,
                        frequency_penalty=0,
                        n=1,
                        stream=False,
                        stop='```')
print(ans)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "gl-b-6"
    }
  ],
  "created": 1673908553,
  "id": "cmpl-6ZSMzxo5cFkXbZSG0bpdLKN2i4H5E",
  "model": "davinci-codex-002-msft",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 5,
    "prompt_tokens": 2258,
    "total_tokens": 2263
  }
}


In [68]:
print(table_nl_desc_test[7]['table_text'])
print(table_nl_desc_test[7]['nl_text'])

[HEAD]: Game|Day|Date|Kickoff|Opponent|Results
Score|Results
Record|Location|Attendance
---------------------------------------------------------------------------------------
[ROW] 1: 1|Sunday|November 10|3:05pm|at Las Vegas Legends|L 3–7|0–1|Orleans Arena|1,836
[ROW] 2: 2|Sunday|November 17|1:05pm|Monterrey Flash|L 6–10|0–2|UniSantos Park|363
[ROW] 3: 3|Saturday|November 23|7:05pm|at Bay Area Rosal|W 10–7|1–2|Cabernet Indoor Sports|652
[ROW] 4: 4|Sunday|December 1|1:05pm|Ontario Fury|W 18–4|2–2|UniSantos Park|207
[ROW] 5: 5|Saturday|December 14|7:05pm|at Sacramento Surge|W 7–6 (OT)|3–2|Estadio Azteca Soccer Arena|215
[ROW] 6: 6|Sunday|December 15|6:00pm|at Bay Area Rosal|L 8–9 (OT)|3–3|Cabernet Indoor Sports|480
[ROW] 7: 7|Sunday|December 22|1:05pm|Turlock Express|W 16–8|4–3|UniSantos Park|218
[ROW] 8: 8|Saturday|January 4|7:05pm|at Ontario Fury|L 5–12|4–4|Citizens Business Bank Arena|2,653
[ROW] 9: 9|Sunday|January 5|1:05pm|San Diego Sockers|L 7–12|4–5|UniSantos Park|388
[ROW] 10: 1